In [83]:
import pandas as pd
import os

In [3]:
faulty_excel = 'S2_Faulty.xlsx'
current_folder_path = os.getcwd()
parent_folder_path = os.path.dirname(current_folder_path)
excel_file_path_F = os.path.join(parent_folder_path, 'data', faulty_excel)

In [5]:
df = pd.read_excel(excel_file_path_F)

In [162]:
df = pd.read_feather("df_f_feather")

In [163]:
df = df.drop(columns = {'time'})

In [164]:
df = df.drop(index=0)

In [92]:
import pandas as pd

# Clean the spaces in the label names
df['label'] = df['label'].str.replace(' ', '')

# Split the dataset based on labels
labels = ['Bias', 'Drift', 'Gain', 'NoFault', 'Outliers', 'Precisiondegradation']

datasets = {}

for label in labels:
    datasets[label] = df[df['label'] == label]

    # Rename the dataset as df_labelname
    globals()[f'df_{label}'] = datasets[label]

    # Optional: Save the dataset to a separate CSV file
    datasets[label].to_csv(f'df_{label}.csv', index=False)


In [24]:
# Print the size of each new DataFrame
print("Size of df_Bias:", df_Bias.shape)
print("Size of df_Drift:", df_Drift.shape)
print("Size of df_Gain:", df_Gain.shape)
print("Size of df_NoFault:", df_NoFault.shape)
print("Size of df_Outliers:", df_Outliers.shape)
print("Size of df_PrecisionDegredatation:", df_Precisiondegradation.shape)

Size of df_Bias: (184700, 2)
Size of df_Drift: (184701, 2)
Size of df_Gain: (184701, 2)
Size of df_NoFault: (184177, 2)
Size of df_Outliers: (524, 2)
Size of df_PrecisionDegredatation: (0, 2)


In [146]:
df_Precisiondegradation = pd.read_csv("df_Precisiondegradation.csv")

In [148]:
df_Precisiondegradation = df_Precisiondegradation.drop(columns={'time'})

In [84]:
import pandas as pd
import numpy as np

def split_column_into_rows(df, num_columns):
    values = df['value'].tolist()  # Get the values from the specified column as a list

    # Calculate the number of rows needed
    num_rows = len(values) // num_columns
    if len(values) % num_columns != 0:
        num_rows += 1

    # Create a new DataFrame with the specified number of columns
    new_df = pd.DataFrame(np.zeros((num_rows, num_columns)))

    # Iterate over the values and populate the new DataFrame
    for i, value in enumerate(values):
        row = i // num_columns
        col = i % num_columns
        new_df.iloc[row, col] = value

    return new_df


In [34]:
new_df = split_column_into_rows(df_Bias, 150)

In [59]:
import pandas as pd
import numpy as np

def split_column_into_rows(df, num_columns, overlapping_ratio):
    values = df['value'].to_numpy()  # Get the values from the specified column as a NumPy array

    # Calculate the number of rows needed
    num_rows = len(values) // num_columns
    if len(values) % num_columns != 0:
        num_rows += 1

    # Calculate the number of overlapping elements between adjacent rows
    overlapping_elements = int(num_columns * overlapping_ratio / 100)

    # Calculate the total number of elements in each row, considering the overlapping elements
    row_elements = num_columns - overlapping_elements

    # Calculate the required number of values to fill the last row
    last_row_fill = num_columns - (num_rows * row_elements)

    # Create a new NumPy array to store the split values
    new_values = np.zeros((num_rows, num_columns))

    # Iterate over the values and populate the new array
    for i, value in enumerate(values):
        row = i // row_elements
        col_start = (i % row_elements) * (row_elements + overlapping_elements)
        col_end = col_start + row_elements

        new_values[row, col_start:col_end] = value

    # Fill the remaining spaces in the last row with zeros
    if last_row_fill > 0:
        new_values[-1, -last_row_fill:] = 0

    # Convert the new array into a DataFrame
    new_df = pd.DataFrame(new_values)

    return new_df


In [66]:
import pandas as pd
import numpy as np

def split_column_into_rows(df, num_columns, overlapping_ratio):
    values = df['value'].to_numpy()  # Get the values from the specified column as a NumPy array

    # Calculate the number of rows needed
    num_rows = len(values) // num_columns
    if len(values) % num_columns != 0:
        num_rows += 1

    # Calculate the number of overlapping elements between adjacent rows
    overlapping_elements = int(num_columns * overlapping_ratio / 100)

    # Calculate the total number of elements in each row, considering the overlapping elements
    row_elements = num_columns - overlapping_elements

    # Calculate the required number of values to fill the last row
    last_row_fill = num_columns - (len(values) % num_columns)

    # Extend the values array with zeros if necessary to ensure even splitting
    values = np.pad(values, (0, last_row_fill), mode='constant', constant_values=0)

    # Reshape the values array into the desired shape with overlapping
    new_values = np.zeros((num_rows, num_columns))

    for i, value in enumerate(values):
        row = i // row_elements
        col_start = (i % row_elements) * (row_elements + overlapping_elements)
        col_end = col_start + row_elements
        if row < num_rows:
            new_values[row, col_start:col_end] = value

    # Convert the new array into a DataFrame
    new_df = pd.DataFrame(new_values)

    return new_df


In [152]:
import pandas as pd
import numpy as np

def split_column_into_rows(df, num_columns, overlapping_ratio):
    values = df['value'].to_numpy()  # Get the values from the specified column as a NumPy array

    # Calculate the number of overlapping elements between adjacent rows
    overlapping_elements = int(num_columns * overlapping_ratio / 100)

    # Calculate the total number of elements in each row, considering the overlapping elements
    row_elements = num_columns - overlapping_elements

    # Calculate the number of rows needed
    num_rows = (len(values) + row_elements - 1) // row_elements

    # Calculate the required number of values to fill the last row
    last_row_fill = (num_rows * num_columns) - len(values)

    # Extend the values array with zeros if necessary to ensure even splitting
    values = np.pad(values, (0, last_row_fill), mode='constant', constant_values=0)

    # Reshape the values array into the desired shape with overlapping
    new_values = np.zeros((num_rows, num_columns))
    for i in range(num_rows):
        start = i * row_elements
        end = start + num_columns
        new_values[i] = values[start:end]

    # Convert the new array into a DataFrame
    new_df = pd.DataFrame(new_values)

    label_name = df.at[0, 'label']
    new_df.insert(0,'label',label_name)

    return new_df


In [165]:
df

,value,label
1,-0.900545,Bias
2,-0.899803,Bias
3,-0.902359,Bias
4,-0.899715,Bias
5,-0.901006,Bias
...,...,...
923500,-1.002532,No Fault
923501,-1.001433,No Fault
923502,-1.001267,No Fault
923503,-0.999077,No Fault


In [173]:
df_Precisiondegradation

,value,label
0,-1.003319,Precisiondegradation
1,-1.005235,Precisiondegradation
2,-1.000776,Precisiondegradation
3,-1.008916,Precisiondegradation
4,-1.000741,Precisiondegradation
...,...,...
184696,-1.001516,Precisiondegradation
184697,-0.995759,Precisiondegradation
184698,-1.001589,Precisiondegradation
184699,-1.001919,Precisiondegradation


In [153]:
new_df1 = split_column_into_rows(df_Precisiondegradation, 10,50 )


In [156]:
def convert_labels_to_integers(dataframe):
    dataframe['label'] = pd.factorize(dataframe['label'])[0] + 1
    return dataframe

In [171]:
def convert_labels_to_integers(dataframe):
    unique_labels = dataframe['label'].unique()
    label_to_integer = {label: i+1 for i, label in enumerate(unique_labels)}
    dataframe['label'] = dataframe['label'].map(label_to_integer)
    
    for label, integer in label_to_integer.items():
        print(f"Label '{label}' changed to integer '{integer}'")
    
    return dataframe

In [168]:
df

,value,label
1,-0.900545,Bias
2,-0.899803,Bias
3,-0.902359,Bias
4,-0.899715,Bias
5,-0.901006,Bias
...,...,...
923500,-1.002532,No Fault
923501,-1.001433,No Fault
923502,-1.001267,No Fault
923503,-0.999077,No Fault


In [172]:
new_df2 = convert_labels_to_integers(df)
new_df2

Label 'Bias' changed to integer '1'
Label 'Drift' changed to integer '2'
Label 'Gain' changed to integer '3'
Label 'Precision degradation' changed to integer '4'
Label 'No Fault' changed to integer '5'
Label 'Outliers' changed to integer '6'


,value,label
1,-0.900545,1
2,-0.899803,1
3,-0.902359,1
4,-0.899715,1
5,-0.901006,1
...,...,...
923500,-1.002532,5
923501,-1.001433,5
923502,-1.001267,5
923503,-0.999077,5
